In [3]:
import logging # Facilitates logging 
import requests # Facilitates making http requests
import json # Facilitates handling of json objects
import get_records # Facilitates reading phones from the restaurant requests table
import sqlalchemy as db
import psycopg2
import pandas as pd

In [30]:
import importlib as imp
get_records = imp.reload(get_records)

In [4]:
database_fields_level_1 = [
        'id',
        'name',
        'phone',
        'price',
        'review_count',
        'rating',
        'categories',
        'is_closed'
        ]

database_fields_level_2 = [
        'country',
        'state',
        'city',
        'zip_code',
        'address1'
        ]

database_fields_level_3 = [
        'longitude',
        'latitude'
        ]

In [20]:
create_status_update_table_statement = """
CREATE TEMPORARY TABLE status_update (row_id INTEGER, request_status VARCHAR, error_message VARCHAR) 
ON COMMIT DROP
"""

insert_status_update_table_statement = """
INSERT INTO status_update (row_id, request_status, error_message) 
VALUES(%s, %s, %s)
"""

update_restaurants_requests_statement = """
UPDATE restaurants_requests
SET 
    phone_request_status = status_update.request_status,
    phone_error_message = status_update.error_message
FROM status_update
WHERE 
    status_update.row_id = restaurants_requests.row_id;
"""

In [6]:
class Yelp_Match:
    """ This class facilitates requests to Yelp by phone number or address. """

    def __init__(self, api_key, match_type = 'phone'):

        # Make sure the match type is valid
        assert match_type in ('phone', 'address')

        # Initiate instance of a class with API key, logger and urls
        self.api_key = api_key
        self.headers = {'Authorization': 'Bearer %s' % self.api_key}
        if (match_type == 'phone'):
            self.search_url = 'https://api.yelp.com/v3/businesses/search/phone'
        self.logger = logging.getLogger(__name__)

    def set_output_folder_path(self, output_folder_path):
        # Set the folder path for output files
        self.output_folder_path = output_folder_path
        self.logger.debug('Output folder set to ' + output_folder_path)

    def search_by_phone(self, phone):
        # Send the phone number request
        params = {'phone' : phone}
        response = requests.get(url = self.search_url,
                params = params,
                headers = self.headers
                )
        self.json = response.json()
        return self.json

    def json_to_file(self, item_number):
        # Write json response to disk
        output_file_path = (self.output_folder_path +
                'item' + item_number + '.json')
        with open(output_file_path, 'w+') as output_file:
            json.dump(self.json, output_file)
            self.logger.debug('Information for item ' + item_number + ' obtained.')
        return None

In [7]:
def request_phone_save(row):
    """ This function makes a request and parses response based on phone. """

    item_number = row['row_id']
    sg_id = row['sname_place_id']
    phone = row['r_phone_number']
    response = searcher.search_by_phone(phone)
    searcher.json_to_file(str(item_number))

    if (not ('businesses' in response)):
        # If no businesses in response mark result as no success
        values = {}
        values['row_id'] = item_number
        values['sname_place_id'] = sg_id
        values['request_status'] = 'zero'
        values['error_message'] = None
        if 'error' in response:
            # If an error in response, save error message
            values['request_status'] = 'error'
            values['error_message'] = response['error']['description']
        for f in database_fields_level_1:
            values[f] = None
        for f in database_fields_level_2:
            values[f] = None
        for f in database_fields_level_3:
            values[f] = None
        return pd.DataFrame([values])

    if (len(response['businesses']) == 0):
        values = {}
        values['row_id'] = item_number
        values['sname_place_id'] = sg_id
        values['request_status'] = 'zero'
        values['error_message'] = None
        for f in database_fields_level_1:
            values[f] = None
        for f in database_fields_level_2:
            values[f] = None
        for f in database_fields_level_3:
            values[f] = None
        return pd.DataFrame([values])

    # Parse the businesses in responses
    businesses = []
    for b in response['businesses']:
        values = {}
        values['row_id'] = item_number
        values['sname_place_id'] = sg_id
        values['request_status'] = 'success'
        values['error_message'] = None
        for f in database_fields_level_1:
            # Get the level - 1 fields
            if (f == 'price'):
                try:
                    values[f] = len(b[f])
                except KeyError:
                    values[f] = -1
                except:
                    logger.error('Error in price handling. ', exc_info = True) 
            else:
                values[f] = b[f]
        for f in database_fields_level_2:
            # Get the level - 2 fields
            values[f] = b['location'][f]
        for f in database_fields_level_3:
            # Get the level - 3 fields
            values[f] = b['coordinates'][f]
        businesses.append(values)
    return pd.DataFrame(businesses)

In [8]:
api_key = '***REMOVED***'
n_records = 15
output_folder_path = '/home/user/projects/urban/data/input/Yelp/phone_address/phone/'
log_file_path = '/home/user/projects/urban/code/yelp-more-download/logs/yelp_phone_requests.log'

In [9]:
# Inititate an instance of a Yelp_Match class
searcher = Yelp_Match(api_key, match_type = 'phone')
searcher.set_output_folder_path(output_folder_path)

In [10]:
# Set the logging options
logging.basicConfig(level=logging.DEBUG)
logger = searcher.logger
logger.setLevel(logging.INFO)
# Write log to file (DEBUG)
fh = logging.FileHandler(log_file_path)
fh.setLevel(logging.INFO)
# Write log to console (ERROR)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
# Create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
ch.setFormatter(formatter)
# Add the handlers to the logger
logger.addHandler(fh)
logger.addHandler(ch)

In [11]:
logger.info('Making requests based.')

2019-12-14 13:03:47,906 - __main__ - INFO - Making requests based.
INFO:__main__:Making requests based.


In [71]:
# Connect to the database via SQLalchemy
engine = db.create_engine('postgresql://{user}:{user_pass}@{host}/{dataname1}')
connection = engine.connect()

# Connect via psycopg2
conn = psycopg2.connect('dbname=dataname1 user={user} password={user_pass}')

In [72]:
requests_table = get_records.get_request_phones(n_records, engine, conn)

In [21]:
requests_table.loc[1, 'r_phone_number'] = '+170498T32760'

In [73]:
requests_table

,row_id,sname_place_id,r_phone_number,r_location_name,r_zip_code,r_street_address,r_city,r_state,y_id,phone_request_status,address_request_status,phone_error_message,address_error_message
0,22,sg:8ac66769f353436bbf82ee45e5db1f24,+15159869182,McDonald's,50111,101 gateway drive,grimes,ia,None,success,needed,None,None
1,24,sg:39abe1ba075243fa8a51ee8d256bd7f6,+17634948844,Highlander,55311,7801 county road 101 rush creek golf club,maple grove,mn,None,success,needed,None,None
2,27,sg:d7ff7fbdd361445992c009d82edf901a,+14054701261,Industry Gastro Lounge,73142,2800 north west 140th street,oklahoma city,ok,None,success,needed,None,None
3,46,sg:3c22536af466408fa4019fb65e5bb44b,+19077477827,Subway,99835,327 seward street,sitka,ak,5FutJzRMyhdPdzmQUIJdxg,success,needed,None,None
4,47,sg:7ca8916a836942baa1939493bcba5c4f,+14809213500,Noodles & Company,85281,2000 east rio salado parkway,tempe,az,None,success,needed,None,None


In [74]:
result = requests_table.apply(request_phone_save, axis = 1)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.yelp.com:443
DEBUG:urllib3.connectionpool:https://api.yelp.com:443 "GET /v3/businesses/search/phone?phone=%2B15159869182 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.yelp.com:443
DEBUG:urllib3.connectionpool:https://api.yelp.com:443 "GET /v3/businesses/search/phone?phone=%2B15159869182 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.yelp.com:443
DEBUG:urllib3.connectionpool:https://api.yelp.com:443 "GET /v3/businesses/search/phone?phone=%2B17634948844 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.yelp.com:443
DEBUG:urllib3.connectionpool:https://api.yelp.com:443 "GET /v3/businesses/search/phone?phone=%2B14054701261 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.yelp.com:443
DEBUG:urllib3.connectionpool:https://api.yelp.com:443 "GET /v3/businesses/search/phone?

In [75]:
result = pd.concat(result.values)

In [76]:
result

,row_id,sname_place_id,request_status,error_message,id,name,phone,price,review_count,rating,categories,is_closed,country,state,city,zip_code,address1,longitude,latitude
0,22,sg:8ac66769f353436bbf82ee45e5db1f24,success,None,LhuWY0VAMc3oAMPCzhfncw,McDonald's,+15159869182,1,8,2.5,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",False,US,IA,Grimes,50111,101 Gateway Dr,-93.779335,41.687938
0,24,sg:39abe1ba075243fa8a51ee8d256bd7f6,success,None,V5EQM-PKSYeh3Ly2HHPnSA,Highlander,+17634948844,-1,9,3.0,"[{'alias': 'newamerican', 'title': 'American (...",False,US,MN,Maple Grove,55311,7801 County Road 101,-93.517600,45.097950
0,27,sg:d7ff7fbdd361445992c009d82edf901a,success,None,ZzE5Worwf02UqHoD3r6tWg,Industry Gastro Lounge,+14054701261,2,104,2.5,"[{'alias': 'gastropubs', 'title': 'Gastropubs'...",False,US,OK,Oklahoma City,73142,2800 NW 140th St,-97.565437,35.613757
0,46,sg:3c22536af466408fa4019fb65e5bb44b,success,None,5FutJzRMyhdPdzmQUIJdxg,Subway Restaurants,+19077477827,1,5,3.0,"[{'alias': 'sandwiches', 'title': 'Sandwiches'}]",False,US,AK,Sitka,99835,327 Seward St,-135.334640,57.051530
0,47,sg:7ca8916a836942baa1939493bcba5c4f,success,None,CO8MpD1bSkBPArXKBKkIfA,Noodles & Company,+14809213500,2,160,3.5,"[{'alias': 'noodles', 'title': 'Noodles'}, {'a...",False,US,AZ,Tempe,85281,2000 E Rio Salado Pkwy,-111.901593,33.433435


In [77]:
failures = result.loc[result['request_status'] != 'success']
cur = conn.cursor()
cur.execute(create_status_update_table_statement)
rows = zip(failures.row_id, failures.request_status, failures.error_message)
cur.executemany(insert_status_update_table_statement, rows)
cur.execute(update_restaurants_requests_statement)
conn.commit()

In [78]:
conn.close()

In [79]:
success = result.loc[result['request_status'] == 'success']
success.drop(['request_status', 'error_message'], axis = 1, inplace = True)
success.reset_index(drop = True, inplace = True)
success.to_sql('more_yelp_restaurants',
        con = engine,
        index = False,
        if_exists = 'append',
        dtype = {'categories' : db.types.JSON})

In [80]:
engine.dispose()
conn.close()

In [2]:
############################### Checking address-based requests

import logging # Facilitates logging 
import requests # Facilitates making http requests
import json # Facilitates handling of json objects
import get_addresses # Facilitates reading addresses from the restaurant requests table
import sqlalchemy as db
import psycopg2
import pandas as pd
import sys 

address_limit = 5 

database_fields_level_1 = [ 
        'id',
        'name',
        'phone',
        'price',
        'review_count',
        'rating',
        'categories',
        'is_closed'
        ]   

database_fields_level_2 = [ 
        'country',
        'state',
        'city',
        'zip_code',
        'address1'
        ]   

database_fields_level_3 = [ 
        'longitude',
        'latitude'
        ]   

########################### SQL Statements #####################################

create_status_update_table_statement = """
CREATE TEMPORARY TABLE status_update (row_id INTEGER, request_status VARCHAR, error_message VARCHAR) 
ON COMMIT DROP
"""

insert_status_update_table_statement = """
INSERT INTO status_update (row_id, request_status, error_message) 
VALUES(%s, %s, %s)
"""

update_restaurants_requests_statement = """
UPDATE restaurants_requests
SET 
    address_request_status = status_update.request_status,
    address_error_message = status_update.error_message
FROM status_update
WHERE 
    status_update.row_id = restaurants_requests.row_id;
"""

################################################################################

In [3]:
class Yelp_Match:                                                                                                                                                                                                                             
    """ This class facilitates requests to Yelp by phone number or address. """

    def __init__(self, api_key, match_type = 'address'):

        # Make sure the match type is valid
        assert match_type in ('phone', 'address')

        # Initiate instance of a class with API key, logger and urls
        self.api_key = api_key
        self.headers = {'Authorization': 'Bearer %s' % self.api_key}
        if (match_type == 'phone'):
            self.search_url = 'https://api.yelp.com/v3/businesses/search/phone'
        if (match_type == 'address'):
            self.search_url = 'https://api.yelp.com/v3/businesses/search'
        self.logger = logging.getLogger(__name__)

    def set_output_folder_path(self, output_folder_path):
        # Set the folder path for output files
        self.output_folder_path = output_folder_path
        self.logger.debug('Output folder set to ' + output_folder_path)

    def search_by_phone(self, phone):
        # Send the phone number request
        params = {'phone' : phone}
        response = requests.get(url = self.search_url,
                params = params,
                headers = self.headers
                )
        self.json = response.json()
        return self.json

    def search_by_address(self, name, address):
        # Send the name, address request
        params = {'term': name,
                'location': address,
                'limit': address_limit
                }
        response = requests.get(url = self.search_url,
                params = params,
                headers = self.headers
                )
        self.json = response.json()
        return self.json

    def json_to_file(self, item_number):
        # Write json response to disk
        output_file_path = (self.output_folder_path +
                'item' + item_number + '.json')
        with open(output_file_path, 'w+') as output_file:
            json.dump(self.json, output_file)
            self.logger.debug('Information for item ' + item_number + ' obtained.')
        return None

In [6]:
def request_address_save(row):
    """ This function makes a request and parses response based on address. """

    item_number = row['row_id']
    sg_id = row['sname_place_id']
    name = row['r_location_name']
    street_address = row['r_street_address']
    city = row['r_city']
    state = row['r_state']
    zip_code = row['r_zip_code']
    address = ','.join([street_address, city, state + str(zip_code)])
    response = searcher.search_by_address(name, address)
    searcher.json_to_file(str(item_number))

    if (not ('businesses' in response)):
        # If no businesses in response mark result as no success
        values = {}
        values['row_id'] = item_number
        values['sname_place_id'] = sg_id
        values['request_status'] = 'zero'
        values['error_message'] = None
        if 'error' in response:
            # If an error in response, save error message
            values['request_status'] = 'error'
            values['error_message'] = response['error']['description']
        for f in database_fields_level_1:
            values[f] = None
        for f in database_fields_level_2:
            values[f] = None
        for f in database_fields_level_3:
            values[f] = None
        return pd.DataFrame([values])

    if (len(response['businesses']) == 0):
        values = {}
        values['row_id'] = item_number
        values['sname_place_id'] = sg_id
        values['request_status'] = 'zero'
        values['error_message'] = None
        for f in database_fields_level_1:
            values[f] = None
        for f in database_fields_level_2:
            values[f] = None
        for f in database_fields_level_3:
            values[f] = None
        return pd.DataFrame([values])

    # Parse the businesses in responses
    businesses = []
    for b in response['businesses']:
        values = {}
        values['row_id'] = item_number
        values['sname_place_id'] = sg_id
        values['request_status'] = 'success'
        values['error_message'] = None
        for f in database_fields_level_1:
            # Get the level - 1 fields
            if (f == 'price'):
                try:
                    values[f] = len(b[f])
                except KeyError:
                    values[f] = -1
                except:
                    logger.error('Error in price handling. ', exc_info = True)
            else:
                values[f] = b[f]
        for f in database_fields_level_2:
            # Get the level - 2 fields
            values[f] = b['location'][f]
        for f in database_fields_level_3:
            # Get the level - 3 fields
            values[f] = b['coordinates'][f]
        businesses.append(values)
    return pd.DataFrame(businesses)

In [7]:
api_key = '***REMOVED***'
n_records = 15
output_folder_path = '/home/user/projects/urban/data/input/Yelp/phone_address/address/'
log_file_path = '/home/user/projects/urban/code/yelp-more-download/logs/yelp_address_requests.log'
postfix = 'user.test'

In [8]:
searcher = Yelp_Match(api_key, match_type = 'address')                                                             
searcher.set_output_folder_path(output_folder_path) 

In [9]:
searcher.output_folder_path

'/home/user/projects/urban/data/input/Yelp/phone_address/address/'

In [10]:
engine = db.create_engine('postgresql://{user}:{user_pass}@{host}/{dataname1}')
connection = engine.connect()
# Connect via psycopg2
conn = psycopg2.connect('dbname=dataname1 user={user} password={user_pass}')

In [11]:
requests_table = get_addresses.get_request_addresses(n_records, engine, conn, postfix)

In [12]:
result = requests_table.apply(request_address_save, axis = 1)

In [13]:
result = pd.concat(result.values)

In [22]:
result.loc[result['row_id']==289773]

,row_id,sname_place_id,request_status,error_message,id,name,phone,price,review_count,rating,categories,is_closed,country,state,city,zip_code,address1,longitude,latitude
0,289773,sg:c68112ed2e9c48c9ac38960a1083b314,success,None,1fDy1XKuKk3tUtLOPqsc9w,Blueberry Cafe,+19378485900,2,157,4.0,"[{'alias': 'breakfast_brunch', 'title': 'Break...",False,US,OH,Bellbrook,45305,72 Bellbrook Plz,-84.073382,39.636978
1,289773,sg:c68112ed2e9c48c9ac38960a1083b314,success,None,oVk4pYuxR99qbJRutGIgkQ,Sleepy Bee Cafe - Blue Ash,+15132412339,2,378,4.0,"[{'alias': 'breakfast_brunch', 'title': 'Break...",False,US,OH,Blue Ash,45242,9514 Kenwood Rd,-84.376370,39.230220
2,289773,sg:c68112ed2e9c48c9ac38960a1083b314,success,None,s47ADFrjRnmjvxvNdm1tTA,Coffee Break Roasting,+15138411100,-1,2,5.0,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",False,US,OH,Cincinnati,45237,1940 Losantiville Ave,-84.454018,39.191826
3,289773,sg:c68112ed2e9c48c9ac38960a1083b314,success,None,_sE5_fnXM7SVcyP_KQurSg,Sleepy Bee Cafe - Oakley,+15135332339,2,654,4.0,"[{'alias': 'breakfast_brunch', 'title': 'Break...",False,US,OH,Cincinnati,45209,3098 Madison Rd,-84.430038,39.153536
4,289773,sg:c68112ed2e9c48c9ac38960a1083b314,success,None,NTEpdLgNDc2XsSHeOdaLcw,Half Day Cafe,+15138212323,2,218,4.0,"[{'alias': 'breakfast_brunch', 'title': 'Break...",False,US,OH,Cincinnati,45215,1 Wyoming Ave,-84.472382,39.227081


In [23]:
success = result.loc[result['request_status'] == 'success'].copy()

In [24]:
success

,row_id,sname_place_id,request_status,error_message,id,name,phone,price,review_count,rating,categories,is_closed,country,state,city,zip_code,address1,longitude,latitude
0,224158,sg:7e69872209114effb165de63ea2637c4,success,None,G0EVYeOpY0ePjhCfaZl9kg,Bap And Chicken,+16513330929,2,60,3.5,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",False,US,MN,Saint Paul,55105,1328 Grand Ave,-93.156327,44.939920
1,224158,sg:7e69872209114effb165de63ea2637c4,success,None,K77trgKuz93SW47MWPQiSQ,Hoban Korean Restaurant,+16516883447,2,381,3.5,"[{'alias': 'korean', 'title': 'Korean'}]",False,US,MN,Eagan,55122,1989 Silver Bell Rd,-93.205380,44.820701
2,224158,sg:7e69872209114effb165de63ea2637c4,success,None,qjubYib8lGtDENeOZmWOVA,Hoban Korean BBQ,+16123457214,2,240,3.5,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",False,US,MN,Minneapolis,55408,2939 Hennepin Ave S,-93.297890,44.948780
3,224158,sg:7e69872209114effb165de63ea2637c4,success,None,vi58lz0GLWITkg_r0WieNg,Kbop Korean Bistro,+16123314993,1,226,4.5,"[{'alias': 'korean', 'title': 'Korean'}]",False,US,MN,Minneapolis,55414,425 13th Ave SE,-93.236773,44.981967
4,224158,sg:7e69872209114effb165de63ea2637c4,success,None,HkmMKv8T0GmAiVNtL6PiEA,Popeyes Louisiana Kitchen,+19524176678,1,10,2.5,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",False,US,MN,Bloomington,55425,388 South Ave,-93.241397,44.853966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,290166,sg:c684030d51294517a05f8ea1085fef61,success,None,cASnoLvllpiS-ivQoDaPHw,Dairy Queen Orange Julius,+15614294530,1,9,3.5,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",False,US,FL,West Palm Beach,33405,7900 S Dixie Hwy,-80.055860,26.646670
1,290166,sg:c684030d51294517a05f8ea1085fef61,success,None,lkSbEZiYhQDOkXe5KOR66g,Carvel,+15615887853,1,30,4.0,"[{'alias': 'icecream', 'title': 'Ice Cream & F...",False,US,FL,West Palm Beach,33405,5901 S Dixie Hwy,-80.056037,26.661514
2,290166,sg:c684030d51294517a05f8ea1085fef61,success,None,6HHrLTFJF4WHvNvmKvaG4g,Sloan's Ice Cream,,2,99,4.0,"[{'alias': 'icecream', 'title': 'Ice Cream & F...",False,US,FL,West Palm Beach,33401,700 S Rosemary Ave,-80.055875,26.707666
3,290166,sg:c684030d51294517a05f8ea1085fef61,success,None,Vw0X8Z-p3u0BjWOyBl_V_w,La Michoacana Natural,,1,23,4.5,"[{'alias': 'icecream', 'title': 'Ice Cream & F...",False,US,FL,Lake Worth,33461,3881 S Congress Ave,-80.089330,26.619190


In [25]:
success.drop(['request_status', 'error_message'], axis = 1, inplace = True)

In [26]:
success.reset_index(drop = True, inplace = True)

In [27]:
success.to_sql('address_yelp_restaurants',                                                                                                                                                                                                
    con = engine,
    index = False,
    if_exists = 'append',
    dtype = {'categories' : db.types.JSON})

In [28]:
# End database connection
    engine.dispose()

In [29]:
status_update = result.copy()
status_update.drop_duplicates(subset = 'row_id', keep = 'first', inplace = True)

In [35]:
status_update

,row_id,sname_place_id,request_status,error_message,id,name,phone,price,review_count,rating,categories,is_closed,country,state,city,zip_code,address1,longitude,latitude
0,224158,sg:7e69872209114effb165de63ea2637c4,success,None,G0EVYeOpY0ePjhCfaZl9kg,Bap And Chicken,+16513330929,2,60,3.5,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",False,US,MN,Saint Paul,55105,1328 Grand Ave,-93.156327,44.939920
0,249862,sg:c073c0925bb642158561e3f8fadafecd,success,None,yBjzMJdUHiSc_EQn0W6S_A,Zaxby's Chicken Fingers & Buffalo Wings,+17068128993,1,12,2.5,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",False,US,GA,Lagrange,30241,1488 Lafayette Pkwy,-84.978518,33.041910
0,250036,sg:7c349c0a85a64b36a3fe3ff5ef9cb86c,success,None,yFoowyYPRrT4Th-hr6ioAA,Zaxby's Chicken Fingers & Buffalo Wings,+12053384822,-1,8,2.0,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",False,US,AL,Pell City,35125,280 Vaughan Ln,-86.274890,33.607560
0,261598,sg:eefa90d941ee4087a5fff4686dba4c8a,success,None,StT3KC4uvGr9MASqqTniCg,Lunch Box,+13372338298,1,9,3.5,"[{'alias': 'newamerican', 'title': 'American (...",False,US,LA,Lafayette,70507,4302 Moss St,-92.008105,30.279253
0,275248,sg:98a5ccd2df5049398c45bb5315aef0f3,success,None,Ohy6TtDSeJkVAtIHXP7pMw,Subway Restaurants,+17027226430,1,21,3.0,"[{'alias': 'sandwiches', 'title': 'Sandwiches'}]",False,US,NV,Las Vegas,89169,3900 Paradise Road,-115.154355,36.118116
0,287806,sg:b645f80dc0614f16a3d8c30646ed1b1f,success,None,VBD1rrRQkXEv9w5A0tW_SQ,Le Cellier Steakhouse,+14079393463,3,646,4.0,"[{'alias': 'steak', 'title': 'Steakhouses'}]",False,US,FL,Orlando,32830,1510 N Ave,-81.551586,28.371637
0,287953,sg:13afa391dae6450fb09893f10adcd9d7,success,None,Po5rIez-KVQScW8-8hO9fg,Subway Restaurants,+14099435430,1,4,3.0,"[{'alias': 'sandwiches', 'title': 'Sandwiches'}]",False,US,TX,Texas City,77590,2920 Palmer Hwy,-94.938217,29.395662
0,288103,sg:13d19d27822646ec9aa33dd51950757a,success,None,89Td0uei2jBuXfXshiQYMg,Wendy's,+18328691045,1,11,1.5,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",False,US,TX,Katy,77493,1484 Katy Fort Bend Rd,-95.800901,29.793710
0,288153,sg:b6477536fea142d3aefe13e55e13b59b,zero,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,NaN,NaN
0,288845,sg:b621daf0d0e8439c84bb1fe7158cd328,success,None,ULprca3gDzaNqmD8IFwAww,Meadow Grill,,2,21,3.5,"[{'alias': 'burgers', 'title': 'Burgers'}]",False,US,CA,Yosemite National Park,95389,Curry Village,-119.572271,37.737873


In [36]:
cur = conn.cursor()
cur.execute(create_status_update_table_statement)
rows = zip(status_update.row_id, status_update.request_status, status_update.error_message)
cur.executemany(insert_status_update_table_statement, rows)
cur.execute(update_restaurants_requests_statement)
conn.commit()
conn.close()